In [73]:
# IMPORTO LIBRERIAS
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [97]:
# DEFINO LA FUNCION QUE USO PARA LIMPIAR EL DATASET DEL CLIMA

def clean_clima(df):
    #df.drop([0], axis=0, inplace=True)
    df.dropna(inplace=True)
    df['HORA'] = df['HORA'].astype(int)
    df['TEMP'] = df['TEMP'].astype(float)
    df['HUM'] = df['HUM'].astype(float)
    df['PNM'] = df['PNM'].astype(float)
    df['DD'] = df['DD'].astype(float)
    df['FF'] = df['FF'].astype(float)
    df_fechas = df['FECHA'].str.split(pat='-',n=2,expand=True)
    df_fechas['año'] = df_fechas[2].map(str) + '-' + df_fechas[1].map(str) + '-' + df_fechas[0].map(str)
    df_fechas.drop(columns=[0,1,2], inplace=True)
    df.drop(columns=['FECHA'],inplace=True)
    df = pd.concat([df,df_fechas], axis=1)
    df['HORA'] = df['HORA'].astype(int)
    df['fecha-hora'] = df.año + ' ' + df.HORA.map(str)
    df = df[df['NOMBRE'] == 'AEROPARQUE AERO']
    return df

In [86]:
# CARGO DATASETS DE CALIDAD DE AIRE Y FLUJO VEHICULAR
aq = pd.read_csv(r'C:\Users\Ezequiel Vannucchi\Desktop\UTN\Cs de datos\DataSets\calidad-de-aire-2019.csv')
fv = pd.read_csv(r'C:\Users\Ezequiel Vannucchi\Desktop\UTN\Cs de datos\DataSets\flujo-vehicular-por-radares-2019.csv')

In [87]:
# FILTRO EL DATASET FLUJO VEHICULAR POR LA ESTACION DE LA BOCA
fv_boca = fv[fv['disp_nombre'] == 'RD134 Ituzaingo']
fv_boca.head()
fv_boca.tail()

,Unnamed: 0,fecha,hora,autopista_nombre,disp_nombre,disp_ubicacion,seccion_sentido,lat,long,cantidad
140989,140989,2019-08-30,21,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,B,-34.6326,-58.37861,3673
141016,141016,2019-08-30,22,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,A,-34.6326,-58.37861,2184
141017,141017,2019-08-30,22,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,B,-34.6326,-58.37861,3006
141044,141044,2019-08-30,23,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,A,-34.6326,-58.37861,1648
141045,141045,2019-08-30,23,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,B,-34.6326,-58.37861,2369


In [88]:
# CREO EL CAMPO FECHA - HORA PARA PODER UNIRLO CON EL DE CALIDAD DE AIRE
fv_boca['fecha-hora'] = fv_boca.fecha + ' ' + fv_boca.hora.map(str)
fv_boca.head()

C:\Users\Ezequiel Vannucchi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Unnamed: 0,fecha,hora,autopista_nombre,disp_nombre,disp_ubicacion,seccion_sentido,lat,long,cantidad,fecha-hora
16,16,2019-01-01,24,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,A,-34.6326,-58.37861,504,2019-01-01 24
17,17,2019-01-01,24,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,B,-34.6326,-58.37861,413,2019-01-01 24
39,39,2019-01-01,1,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,A,-34.6326,-58.37861,2397,2019-01-01 1
40,40,2019-01-01,1,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,B,-34.6326,-58.37861,1771,2019-01-01 1
62,62,2019-01-01,2,AU 9 de Julio Sur,RD134 Ituzaingo,2.43,A,-34.6326,-58.37861,3013,2019-01-01 2


In [89]:
# PIVOTEO LA TABLA PARA OBTENER LOS VALORES DE AMBOS FLUJOS
fv_boca_pvt = fv_boca.pivot_table(values='cantidad',index='fecha-hora',columns='seccion_sentido',aggfunc='sum')
fv_boca_pvt.rename(columns= {'A': 'Flujo en Sentido A', 'B': 'Flujo en Sentido B'}, inplace=True)
fv_boca_pvt.head()

seccion_sentido,Flujo en Sentido A,Flujo en Sentido B
fecha-hora,,
2019-01-01 1,2397,1771
2019-01-01 10,668,569
2019-01-01 11,1082,1120
2019-01-01 12,1601,1796
2019-01-01 13,1746,2202


In [90]:
fv_boca_pvt.shape

(4993, 2)

In [91]:
aq.head()

,fecha,hora,co_centenario,no2_centenario,pm10_centenario,co_cordoba,no2_cordoba,pm10_cordoba,co_la_boca,no2_la_boca,pm10_la_boca
0,2019-01-01,1,0.46,7,19,0.36,11,s/d,0.48,10,23
1,2019-01-01,2,0.47,9,19,0.41,13,s/d,0.46,11,23
2,2019-01-01,3,0.48,11,19,0.46,12,s/d,0.46,12,23
3,2019-01-01,4,0.48,16,19,0.50,10,s/d,0.46,11,24
4,2019-01-01,5,0.46,10,19,0.54,10,s/d,0.45,9,24


In [92]:
# EN EL DATASET DE CALIDAD DE AIRE DROPERO LAS COLUMNAS QUE NO ME INTERESAN
aq['fecha-hora'] = aq.fecha + ' ' + aq.hora.map(str)
aq_boca = aq.drop(columns=['co_centenario','no2_centenario', 'pm10_centenario','co_cordoba','no2_cordoba','pm10_cordoba'])
print(aq_boca.head())
print(aq_boca.shape)

        fecha  hora co_la_boca no2_la_boca pm10_la_boca    fecha-hora
0  2019-01-01     1       0.48          10           23  2019-01-01 1
1  2019-01-01     2       0.46          11           23  2019-01-01 2
2  2019-01-01     3       0.46          12           23  2019-01-01 3
3  2019-01-01     4       0.46          11           24  2019-01-01 4
4  2019-01-01     5       0.45           9           24  2019-01-01 5
(2868, 6)


In [93]:
# UNO LAS TABLAS FLUJO VEHICULAR Y CALIDAD DE AIRE BASANDOME EN LA FECHA-HORA

aq_fv = aq_boca.merge(fv_boca_pvt, how='left',on='fecha-hora')
aq_fv.head(10)

,fecha,hora,co_la_boca,no2_la_boca,pm10_la_boca,fecha-hora,Flujo en Sentido A,Flujo en Sentido B
0,2019-01-01,1,0.48,10,23,2019-01-01 1,2397.0,1771.0
1,2019-01-01,2,0.46,11,23,2019-01-01 2,3013.0,2384.0
2,2019-01-01,3,0.46,12,23,2019-01-01 3,2098.0,1835.0
3,2019-01-01,4,0.46,11,24,2019-01-01 4,1165.0,1254.0
4,2019-01-01,5,0.45,9,24,2019-01-01 5,1176.0,1026.0
5,2019-01-01,6,0.44,8,24,2019-01-01 6,952.0,1000.0
6,2019-01-01,7,0.44,9,25,2019-01-01 7,695.0,832.0
7,2019-01-01,8,0.42,9,24,2019-01-01 8,533.0,597.0
8,2019-01-01,9,0.43,8,24,2019-01-01 9,518.0,482.0
9,2019-01-01,10,0.43,8,24,2019-01-01 10,668.0,569.0


In [98]:
# CARGO EL PRIMER DATASET DEL CLIMA PARA INICIAR LA TABLA CLIMA Y DROPEO LA PRIMER COLUMNA
df_clima_tot = pd.read_fwf(r'C:\Users\Ezequiel Vannucchi\Desktop\UTN\Cs de datos\DataSets\tp\TOTAL\datohorario20190101.txt')
df_clima_tot.drop([0], axis=0, inplace=True)

#ITERO SOBRE LOS DISTINTOS ARCHIVOS. PARA QUE LOS LEVANTE TODOS. "J" ES EL MES EN LA RUTA E "I" ES EL DIA. 
#LO DIVIDO EN 2 PORQUE HAY NUMEROS QUE VAN CON EL 0 ADELANTE

for j in range (1,5):
    for i in range(1,9):
        k = i + 1
        df_clima = pd.read_fwf(r'C:\Users\Ezequiel Vannucchi\Desktop\UTN\Cs de datos\DataSets\tp\TOTAL\datohorario20190%d0%d.txt' %(j,k))
        df_clima.drop([0], axis=0, inplace=True)
        df_clima_tot = pd.concat([df_clima_tot,df_clima], axis=0)
    for i in range(9,31):
        try:
            k = i + 1
            df_clima = pd.read_fwf(r'C:\Users\Ezequiel Vannucchi\Desktop\UTN\Cs de datos\DataSets\tp\TOTAL\datohorario20190%d%d.txt' %(j,k))
            df_clima.drop([0], axis=0, inplace=True)
            df_clima_tot = pd.concat([df_clima_tot,df_clima], axis=0)
        except:
            pass
    
# "TRY" Y "EXCEPT" ES PARA QUE SI ENCUENTRA UN PROBLEMA MIENTRAS ITERA LO IGNORE Y SIGA ITERANDO. 
# LO PONGO EN EL SEGUNDO PORQUE COMO LOS MESES TIENEN DISTINTA CANTIDAD DE DIAS SI ME BUSCABA EL 30 DE FEBRERO SE CHOTEABA POR EJ
    
print(df_clima_tot.shape)
# APLICO LA FUNCION QUE DEFINI ARRIBA DE TODO
df_clima_tot = clean_clima(df_clima_tot)
print(df_clima_tot.tail())
print(df_clima_tot.shape)

(242702, 8)
    HORA  TEMP   HUM     PNM     DD    FF           NOMBRE         año  \
20    19  14.9  42.0  1019.9  110.0   9.0  AEROPARQUE AERO  2019-04-30   
21    20  14.5  44.0  1020.3  110.0  13.0  AEROPARQUE AERO  2019-04-30   
22    21  14.5  50.0  1020.3   90.0  15.0  AEROPARQUE AERO  2019-04-30   
23    22  14.3  54.0  1020.3   90.0  13.0  AEROPARQUE AERO  2019-04-30   
24    23  15.4  53.0  1019.9   90.0  13.0  AEROPARQUE AERO  2019-04-30   

       fecha-hora  
20  2019-04-30 19  
21  2019-04-30 20  
22  2019-04-30 21  
23  2019-04-30 22  
24  2019-04-30 23  
(2779, 9)


In [99]:
# COMBINO TODO
aq_fv_final = aq_fv.merge(df_clima_tot, how='left', on='fecha-hora')

# SACO LAS COLUMNAS QUE SOBRAN
aq_fv_final.drop(columns=['NOMBRE', 'año', 'HORA'], inplace=True)

# EXPORTO
aq_fv_final.to_csv(r'C:\Users\Ezequiel Vannucchi\Desktop\UTN\Cs de datos\DataSets\tp\dataset_calidad_de_aire.csv')